## Sentence Bert Training

In [1]:
# Imports
import json
import pandas as pd
from datasets import Dataset
import random
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.losses import CachedMultipleNegativesRankingLoss 
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import TripletEvaluator

Matplotlib created a temporary cache directory at C:\Users\BILLZH~1\AppData\Local\Temp\matplotlib-_rb12bol because the default path (C:\Users\Bill Zhu\.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
# load data
path = '../data/'

with open(path+"train-claims.json", "r") as f:
    train_claims = json.load(f)

train_df = pd.DataFrame(train_claims).transpose()

with open(path+"dev-claims.json", "r") as f:
    dev_claims = json.load(f)
dev_df = pd.DataFrame(dev_claims).transpose()

with open(path+"evidence.json", "r") as f:
    evidence = json.load(f)
evidence_df = pd.DataFrame(list(evidence.items()), columns=["key", "value"])

In [4]:
from datasets import Dataset
import random

claims = []
evidence_texts = []
labels = []

evidence_map = dict(evidence)
evidence_ids = list(evidence_map.keys())

for _, info in train_claims.items():
    claim_text = info["claim_text"]
    positive_ids = set(info["evidences"])  # ensure no duplicates

    # === Add all golden (positive) evidences ===
    for eid in positive_ids:
        if eid in evidence_map:
            claims.append(claim_text)
            evidence_texts.append(evidence_map[eid])
            labels.append(1)

    # === Add at least 5 unique negatives ===
    negatives_added = 0
    tried_ids = set()

    while negatives_added < 10:
        neg_id = random.choice(evidence_ids)

        if neg_id in positive_ids or neg_id in tried_ids:
            continue

        tried_ids.add(neg_id)

        if neg_id in evidence_map:
            claims.append(claim_text)
            evidence_texts.append(evidence_map[neg_id])
            labels.append(-1)
            negatives_added += 1

# onvert to HuggingFace Dataset
data = {
    "claim": claims,
    "evidence": evidence_texts,
    "label": labels
}

dataset = Dataset.from_dict(data)

# Confirm
print(dataset)
print(dataset[0])


Dataset({
    features: ['claim', 'evidence', 'label'],
    num_rows: 16402
})
{'claim': 'Not only is there no scientific evidence that CO2 is a pollutant, higher CO2 concentrations actually help ecosystems support more plant and animal life.', 'evidence': 'Higher carbon dioxide concentrations will favourably affect plant growth and demand for water.', 'label': 1}


In [5]:
model1 = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
model2 = SentenceTransformer('all-MiniLM-L6-v2')
model3 = SentenceTransformer('all-MiniLM-L12-v2')

loss1 = MultipleNegativesRankingLoss(model1)
loss2 = MultipleNegativesRankingLoss(model2)
loss3 = MultipleNegativesRankingLoss(model3)

split_dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_ds = split_dataset["train"]
eval_ds = split_dataset["test"]


In [6]:
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="output",
    # Optional training parameters:
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=100,
    run_name="sentence-transformer-training",
)

In [7]:
import torch 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1.to(device)
model2.to(device)
model3.to(device)

trainer1 = SentenceTransformerTrainer(
        model=model1,
        args=args,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        loss=loss1,
    )

# Train the model
trainer1.train()

# Save the model
trainer1.save_model(f"out_models/model_0")

trainer2 = SentenceTransformerTrainer(
       model=model2,
        args=args,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        loss=loss2,
    )

# Train the model
trainer2.train()

# Save the model
trainer2.save_model(f"out_models/model_1")

trainer3 = SentenceTransformerTrainer(
        model=model3,
        args=args,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        loss=loss3,
    )

# Train the model
trainer3.train()

# Save the model
trainer3.save_model(f"out_models/model_2")

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss
100,4.328800,3.189373


KeyboardInterrupt: 

### First Stage Revrieval

In [8]:
model1 = SentenceTransformer('out_models/model_0')
model2 = SentenceTransformer('out_models/model_1')
model3 = SentenceTransformer('out_models/model_2')



In [9]:
embeddings1 = model1.encode(evidence_df['value'].tolist(), show_progress_bar=True, device='cuda')
embeddings2 = model2.encode(evidence_df['value'].tolist(), show_progress_bar=True, device='cuda')
embeddings3 = model3.encode(evidence_df['value'].tolist(), show_progress_bar=True, device='cuda')

Batches:   0%|          | 0/37776 [00:00<?, ?it/s]

Batches:   0%|          | 0/37776 [00:00<?, ?it/s]

Batches:   0%|          | 0/37776 [00:00<?, ?it/s]

In [10]:
claim_embeddings1 = model1.encode(train_df['claim_text'], show_progress_bar=True, device='cuda')
claim_embeddings2 = model2.encode(train_df['claim_text'], show_progress_bar=True, device='cuda')
claim_embeddings3 = model3.encode(train_df['claim_text'], show_progress_bar=True, device='cuda')


C:\Users\Bill Zhu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sentence_transformers\SentenceTransformer.py:649: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentences_sorted = [sentences[idx] for idx in length_sorted_idx]


Batches:   0%|          | 0/39 [00:00<?, ?it/s]

Batches:   0%|          | 0/39 [00:00<?, ?it/s]

Batches:   0%|          | 0/39 [00:00<?, ?it/s]

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

total_embeddings = (embeddings1 + embeddings2 + embeddings3) / 3
claim_embeddings = (claim_embeddings1 + claim_embeddings2 + claim_embeddings3) / 3

similarity_matrix = cosine_similarity(claim_embeddings, total_embeddings)

In [12]:
import numpy as np

top_k = 25
top_indices = np.argsort(-similarity_matrix, axis=1)[:, :top_k]  # sort descending, get top 100 indices

In [13]:
train_df["top_100_evidence"] = [
    evidence_df.iloc[indices]["key"].tolist() for indices in top_indices
]

In [14]:
total_matches = 0
total_gold = 0
total_predicted = 0

for i in range(len(train_df)):
    gold_evidence = train_df["evidences"].iloc[i]
    predicted_evidence = train_df["top_100_evidence"].iloc[i]

    matches = sum(1 for ev in gold_evidence if ev in predicted_evidence)

    total_matches += matches
    total_gold += len(gold_evidence)
    total_predicted += len(predicted_evidence)  # Should be 100 for each if consistent

# Compute precision, recall, and F1-score
precision = total_matches / total_predicted if total_predicted > 0 else 0
recall = total_matches / total_gold if total_gold > 0 else 0
f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1_score:.4f}")

Precision: 0.0679
Recall: 0.5056
F1-score: 0.1197


### Cross Encoder

In [15]:
## setup dataset.
claims = []
evidence_texts = []
labels = []

evidence_map = dict(evidence)
evidence_ids = list(evidence_map.keys())

for row in train_df.itertuples():
    claim_text = row.claim_text
    positive_ids = set(row.evidences)
    negative_ids = set(row.top_100_evidence) - positive_ids

    # === Add all golden (positive) evidences ===
    for eid in positive_ids:
        if eid in evidence_map:
            claims.append(claim_text)
            evidence_texts.append(evidence_map[eid])
            labels.append(1)
    
    for eid in negative_ids:
        if eid in evidence_map:
            claims.append(claim_text)
            evidence_texts.append(evidence_map[eid])
            labels.append(-1)

# onvert to HuggingFace Dataset
data = {
    "claim": claims,
    "evidence": evidence_texts,
    "label": labels
}

dataset = Dataset.from_dict(data)

# Confirm
print(dataset)
print(dataset[0:5])

Dataset({
    features: ['claim', 'evidence', 'label'],
    num_rows: 32738
})
{'claim': ['Not only is there no scientific evidence that CO2 is a pollutant, higher CO2 concentrations actually help ecosystems support more plant and animal life.', 'Not only is there no scientific evidence that CO2 is a pollutant, higher CO2 concentrations actually help ecosystems support more plant and animal life.', 'Not only is there no scientific evidence that CO2 is a pollutant, higher CO2 concentrations actually help ecosystems support more plant and animal life.', 'Not only is there no scientific evidence that CO2 is a pollutant, higher CO2 concentrations actually help ecosystems support more plant and animal life.', 'Not only is there no scientific evidence that CO2 is a pollutant, higher CO2 concentrations actually help ecosystems support more plant and animal life.'], 'evidence': ['Higher carbon dioxide concentrations will favourably affect plant growth and demand for water.', 'At very high conc

In [ ]:
import logging
import traceback

from datasets import load_dataset

from sentence_transformers.cross_encoder import (
    CrossEncoder,
    CrossEncoderModelCardData,
    CrossEncoderTrainer,
    CrossEncoderTrainingArguments,
)
from sentence_transformers.cross_encoder.evaluation import CrossEncoderNanoBEIREvaluator
from sentence_transformers.cross_encoder.losses import CachedMultipleNegativesRankingLoss
from sentence_transformers.cross_encoder.losses import MultipleNegativesRankingLoss


logging.basicConfig(format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO)
train_batch_size = 64
num_epochs = 1
num_rand_negatives = 5  # How many random negatives should be used for each question-answer pair

model = CrossEncoder("cross-encoder/stsb-roberta-base")

loss = MultipleNegativesRankingLoss(model)

split_dataset = dataset.train_test_split(test_size=0.1, seed=42069)
train_ds = split_dataset["train"]
eval_ds = split_dataset["test"]

# evaluator = CrossEncoderNanoBEIREvaluator(
#     dataset_names=["msmarco", "nfcorpus", "nq"],
#     batch_size=train_batch_size,
# )
#evaluator(model)

# Set up training arguments
args = CrossEncoderTrainingArguments(
    # Required parameter:
    output_dir=f"out_models/cross_encoder",
    # Optional training parameters:
    num_train_epochs=num_epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=train_batch_size,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=True,  # Set to True if you have a GPU that supports BF16
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=50,
    logging_first_step=True,
    run_name="run_cross",  # Will be used in W&B if `wandb` is installed
    seed=12,
)

trainer = CrossEncoderTrainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    loss=loss,
    #evaluator=evaluator,
)

trainer.train()

#evaluator(model)

final_output_dir = f"models/run_cross/final"
model.save_pretrained(final_output_dir)



2025-05-16 08:39:45 - Use pytorch device: cuda:0
2025-05-16 08:40:18 - NanoBEIR Evaluation of the model on ['msmarco', 'nfcorpus', 'nq'] dataset:
2025-05-16 08:40:18 - Evaluating NanoMSMARCO_R100
2025-05-16 08:40:18 - CrossEncoderRerankingEvaluator: Evaluating the model on the NanoMSMARCO_R100 dataset:
2025-05-16 08:40:35 - Queries: 50	Positives: Min 1.0, Mean 1.0, Max 1.0	Negatives: Min 99.0, Mean 99.0, Max 99.0
2025-05-16 08:40:35 -          Base  -> Reranked
2025-05-16 08:40:35 - MAP:     48.96 -> 9.21
2025-05-16 08:40:35 - MRR@10:  47.75 -> 6.16
2025-05-16 08:40:35 - NDCG@10: 54.04 -> 10.77
2025-05-16 08:40:35 - 
2025-05-16 08:40:35 - Evaluating NanoNFCorpus_R100
2025-05-16 08:40:35 - CrossEncoderRerankingEvaluator: Evaluating the model on the NanoNFCorpus_R100 dataset:
2025-05-16 08:42:04 - Queries: 50	Positives: Min 1.0, Mean 50.4, Max 463.0	Negatives: Min 54.0, Mean 92.8, Max 100.0
2025-05-16 08:42:04 -          Base  -> Reranked
2025-05-16 08:42:04 - MAP:     26.10 -> 28.06
202

Step,Training Loss,Validation Loss
